# Use of Measured Aerosol Optical Depth and Precipitable Water to Model Clear Sky Irradiance
*presented at: 2017 IEEE PVSC-44*
*by: Mark A. Mikofski, Clifford W. Hansen, William F. Holmgren and Gregory M. Kimball*

## Introduction
Predicting irradiance is crucial for developing solar power systems. Clear sky models are used to predict irradiance based on solar position and atmospheric data. This paper compares clear sky predictions using atmospheric data from ECMWF with irradiance measurments from SURFRAD. This notebook presents the analysis for the paper and presentation given by the authors at the 2017 IEEE PVSC-44 in Washingtion DC June 25-30th.

## Data
Two sources of data are used in this analysis, SURFRAD and ECMWF, each described in the following sections.

### SURFRAD
SURFRAD data from NOAA contains irradiance measurements at sub-hour increments from seven sites accross the United States. The data can be viewed online or downloaded from a NOAA FTP site <ftp://aftp.cmdl.noaa.gov/data/radiation/surfrad/>. Data is organized folders for each site and yearly subfolders containing individual daily files. The `README` in each site folder provides additional information such as the format of the files, types of data in each column and definitions of abbreviations. The first two rows of each daily file contains header information about the file. The first line is the station name and the second line contains the latitude, longitude and elevation.

|station id |tz    |station name  |latitude |longitude |elevation |
|-----------|------|--------------|---------|----------|----------|
|bon        |-6.0  |     Bondville|    40.05|    -88.37|       213|
|tbl        |-7.0  |Table Mountain|    40.13|   -105.24|      1689|
|dra        |-8.0  |   Desert Rock|    36.62|   -116.02|      1007|
|fpk        |-7.0  |     Fort Peck|    48.31|   -105.10|       634|
|gwn        |-6.0  | Goodwin Creek|    34.25|    -89.87|        98|
|psu        |-5.0  |    Penn State|    40.72|    -77.93|       376|
|sxf        |-6.0  |   Sioux Falls|    43.73|    -96.62|       473|

### ECMWF
The European Center for Medium Weather Forecast hosts datasets for the EU Copernicus MACC clearsky model. This data set contains aerosol optical depth and total column water vapor data derived from ground measurments and satelite data for the entire globe from 2003 to 2012.


In [1]:
# imports and settings
import os

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pvlib
import seaborn as sns

%matplotlib inline

sns.set_context('notebook', rc={'figure.figsize': (16, 8)})

In [2]:
# get the "metadata" that contains the timezone, latitude, longitude, elevation and station id for SURFRAD
METADATA = pd.read_csv('metadata.csv', index_col=0)

In [3]:
# get SURFRAD data
DATADIR = 'surfrad'  # folder where SURFRAD data is stored relative to this file
NA_VAL = '-9999.9'  # value SURFRAD uses for missing data
USECOLS = [0, 6, 7, 8, 9, 10, 11, 12]  # omit 1:year, 2:month, 3:day, 4:hour, 5:minute and 13:jday columns
# read 2003 data for Bondville, IL
bon = pd.read_csv(
    os.path.join(DATADIR, 'bon03.csv'),
    index_col=0,
    usecols=[0, 6, 7, 8, 9, 10, 11, 12],
    na_values=NA_VAL)
# append the data from 2003 to 2012 which coincides with the ECMWF data available
for y in xrange(2004, 2013):
    bon = bon.append(pd.read_csv(
        os.path.join(DATADIR, 'bon%s.csv' % str(y)[-2:]),
        index_col=0,
        usecols=[0, 6, 7, 8, 9, 10, 11, 12],
        na_values=NA_VAL
    ))
bon.index.rename('timestamps', inplace=True)

In [ ]:
# get AOD and water vapor for the Bondville, IL site.

In [ ]:
bon.index.name

In [ ]:
bon[['dni', 'dhi', 'ghi']]['2003-06-15':'2003-06-16'].plot()

In [ ]:
%whos

In [ ]:
df[df['qc_dwsolar']==0]

In [ ]:
df.drop('qc_dwsolar',1)

![Creative Commons License](https://i.creativecommons.org/l/by/4.0/88x31.png)
Use of Measured Aerosol Optical Depth and Precipitable Water to Model Clear Sky Irradiance by [Mark A. Mikofski, Clifford W. Hansen, William F. Holmgren and Gregory M. Kimball](https://github.com/mikofski/pvsc44-clearsky-aod) is licensed under a [Creative Commons Attribution 4.0 International License](https://creativecommons.org/licenses/by/4.0/).
